In [1]:
# Date 22 March 2021, Author: Franziska Nitsch
# This notebook implements a class called PDE_model that includes methods that can be used 
# to solve a partial differential equation. 

import numpy as np
from scipy.integrate import odeint 
from scipy.optimize import fsolve

# define method simulate within base class Simulator, so that it can be passed on to the classes
# for the ODE_model and the PDE_model
class Simulator:
    
    def simulate(self, X0, t, input_signal):
        return  odeint(self.RHS, X0, t, args=(input_signal,))

    
# class that defines a simple ordinary differential equation with one external input
# knows the method simulate from base class Simulator
class ODE_model(Simulator):
    
    def RHS(self,x,t,input_signal):
        dxdt = -x + input_signal(t)
        return dxdt    
    
    
# class that defines some variables (length L, number of spatial discretization elements N  
# and dependent variables dx and xx for the spatial grid) for the discretisation of certain partial differential equations (PDE) in 1D   
class PDEdiscretisation(Simulator):
    def __init__(self,N):
        self.N = N
        self.L = 1
    
    @property
    # step size
    def dx(self):
        return self.L/self.N
        
    @property
    # spatial grid
    def xx(self):
        return np.linspace(0,self.L, num = self.N+1)
    


        

# Class for the simulation of a PDE model of gas entering at the bottom of a liquid column and expanding as it propagates towards the top,
# with the gas influx depending on the pressure at the bottom and with the pressure at the top as an external input.
# The PDE (partial differential equation) is described in more detail at the method RHS
# Inherits methods for simulation and discretization from base classes PDEdiscretisation and Simulator
class PDE_model(PDEdiscretisation):
    def __init__(self,N):
        self.friction = 1 # friction coefficient
        self.gravity = 1 # gravity coefficient
        self.velocity1 = 1 # coefficient for velocity profile
        self.velocity2 = .1 # coefficient for velocity profile
        self.expansion = 1 # gas expansion coefficient
        self.q_liquid_influx = 1 # fluid influx rate
        self.inflow_coefficient = 1 # coefficient for gas influx boundary condition: gas_influx = inflow_coefficient*max(0,p0-pressure(bottom boundary))
        self.p0 = 5  # coefficient for gas influx boundary condition, gas only enters if pressure at bottom is greater than p0
        super().__init__(N)
        
        

    
    # for a given pressure on the bottom (p_bottom), calculate the pressure on the top (p_top)
    def pbottom_to_ptop(self, p_bottom, alpha):
        q_gas_influx = self.inflow_coefficient*np.maximum(0,self.p0-p_bottom) # gas inflow at bottom
        q = q_gas_influx + self.q_liquid_influx # total flow
        p = p_bottom
        # calculate p_top via for loop iterating over grid
        for i in range(self.N):
            p = p-self.dx*(self.gravity*(1-alpha[i])+self.friction*q )    
        return p
    
    
    # computes the difference between the actual pressure at the top and pbottom_to_ptop (for use in fsolve)
    def pbottom_to_ptop_zero(self, p_bottom, alpha, p_top_actual):
        return (self.pbottom_to_ptop(p_bottom, alpha)-p_top_actual)
        
    
    # computes the right-hand side of the PDE 
    # alpha_t = - velocity* alpha_x + E (gas expansion)
    # alpha(x=0) = q_gas_influx/(q_gas_influx+q_liquid_influx)
    # q_gas_influx = inflow_coefficient*max(0,p0-pressure(x=0))
    # pressure(x) = pressure_top + integral_x^1 gravity*(1-alpha(y)) + friction*(q_gas_influx+q_liquid_influx) dy
    def RHS(self,alpha,t,input_signal):  
        # using a non linear solver (fsolve), find the p_bottom that solves pbottom_to_ptop_zero.
        p_b_solution = fsolve(self.pbottom_to_ptop_zero,self.p0,args= (alpha, input_signal(t)),xtol=1e-06)

        # determine influx
        q_gas_influx = self.inflow_coefficient*max(0,self.p0-p_b_solution)
        q_total_flow = q_gas_influx + self.q_liquid_influx
        
        # p is the pressure along the fluid column, it can be calculated from top to bottom if the 
        # pressure on the top (ptop) and the flow rate (q_total_flow) are known 
        p = np.zeros(self.N+1)
        p[self.N] = input_signal(t)
        for i in reversed(range(self.N)):
            p[i] = p[i+1]+self.dx*(self.gravity*(1-alpha[i])+self.friction*q_total_flow)
       
        # boundary condition at x=0
        alpha0 = np.array(q_gas_influx/q_total_flow)
        
        # alpha_extended combines the boundary condition alpha0 into the state alpha
        alpha_extended=np.concatenate((np.atleast_1d(alpha0),alpha))
      
        # alpha derivative with respect to x
        alpha_x = np.subtract(alpha_extended[1:(self.N+1)],alpha_extended[0:self.N])/self.dx
       
        # velocity profile and gas expansion term E
        velocity = self.velocity1*(q_total_flow)*np.exp(self.velocity2*self.xx)
        E = np.multiply(alpha,(1-alpha))*self.expansion
        
        # time derivative of alpha
        alpha_t = -np.multiply(velocity[1:self.N+1],alpha_x)+ E
      
        return alpha_t
   
    
    
    
    
        


In [2]:
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
# define an instance of class PDE_model      
my_model = PDE_model(30)


# look at dependent variables
print("N =")
print(my_model.N)
print("L =")
print(my_model.L)
print("dx =")
print(my_model.dx)
print("xx =")
print(my_model.xx)


# two different input signals to see model behaviour (ptop).

def pressure_top_signal(t_):
    if t_ < 5:
        p_top = 2
    else:
        p_top = 2.8
    return p_top

# time dependent input signal
def pressure_top_signal2(t_):
    if t_ < 5:
        p_top = 2.8
    else:
        p_top = 1
    return p_top

# time discretisation
t = np.linspace(0,10,num = 100)

# vector for plotting input signals :
input_signal_1plot = [pressure_top_signal(t_) for t_ in t]
input_signal_2plot = [pressure_top_signal2(t_) for t_ in t]
    




# test pbottom to ptop function
alpha = np.ones(my_model.N)*0.5

# for plotting p_bottom to p_top as a function of pbottom 
p_bottom = np.linspace(0,10,num=100) 
solution1 = np.zeros(len(p_bottom))
solution2 = np.zeros(len(p_bottom))
for i in range(len(p_bottom)):
    solution1[i] = my_model.pbottom_to_ptop_zero(p_bottom[i], alpha, pressure_top_signal(t[1]))
    solution2[i] = my_model.pbottom_to_ptop_zero(p_bottom[i], alpha, pressure_top_signal(t[-1]))  # input signal at last t

# open plots in new window
%matplotlib qt   

plt.figure(0)
plt.plot(p_bottom,solution1,label="p_top=%s" % (pressure_top_signal(t[1])))
plt.plot(p_bottom,solution2,label="p_top=%s" % (pressure_top_signal(t[-1])))
plt.ylabel('p_bottom_to_ptop')
plt.xlabel('p_bottom')
plt.legend()






# initial condition (gas volume fraction along the well):
X0 = np.ones(my_model.N)*0.5

# call simulator with two different input signals
trajectory = my_model.simulate(X0,t,pressure_top_signal)

trajectory2 = my_model.simulate(X0,t,pressure_top_signal2)
# plot the result. The plots will show the model simulation along the 10 points chosen
# for the discretisation along the well. The higher the point, the more gas expansion we see,
# as a consequence of gravity.

plt.figure(1)
plt.subplot(2, 1, 1)
plt.plot(t,input_signal_1plot)
plt.xlabel('time')
plt.ylabel('input signal 1')
plt.title('pressure at the top')
plt.subplot(2, 1, 2)
plt.plot(t,trajectory[:,0::3])
plt.ylabel('gas volume fraction')
plt.xlabel('time')
plt.title('simulated PDE at 10 discretization points')

plt.figure(2)
plt.subplot(2, 1, 1)
plt.plot(t,input_signal_2plot)
plt.xlabel('time')
plt.ylabel('input signal 2')
plt.title('pressure at the top')
plt.subplot(2, 1, 2)
plt.plot(t,trajectory2[:,0::3])
plt.ylabel('gas volume fraction')
plt.xlabel('time')
plt.title('simulated PDE at 10 discretization points')

# surface level plot of gas volume fraction
plot_xx = np.linspace(0, my_model.L, num = my_model.N)
T, X_ = np.meshgrid(t, plot_xx)

fig=plt.figure(3)
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(T, X_, trajectory.transpose(), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.xlabel('time')
plt.ylabel('x')
plt.title('gas volume fraction')


fig=plt.figure(4)
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(T, X_, trajectory2.transpose(), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
plt.xlabel('time')
plt.ylabel('x')
plt.title('gas volume fraction')



N =
30
L =
1
dx =
0.03333333333333333
xx =
[0.         0.03333333 0.06666667 0.1        0.13333333 0.16666667
 0.2        0.23333333 0.26666667 0.3        0.33333333 0.36666667
 0.4        0.43333333 0.46666667 0.5        0.53333333 0.56666667
 0.6        0.63333333 0.66666667 0.7        0.73333333 0.76666667
 0.8        0.83333333 0.86666667 0.9        0.93333333 0.96666667
 1.        ]


/Users/franzi/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


Text(0.5, 0.92, 'gas volume fraction')

In [3]:
import matplotlib.pyplot as plt
import numpy
# initial condition
x0 = 2

# time points
t = numpy.arange(0,40,0.1)

# time dependent input signal
def input_signalODE(t_1):
    if t_1 < 10:
        input_signal = 1
    elif t_1 < 20:
        input_signal = 2
    elif t_1 < 30:
        input_signal = 1.5
    else:
        input_signal = 0
    return input_signal

# define instance of class ODE_model
myode_model = ODE_model()
x = myode_model.simulate(x0,t,input_signalODE)

# plot the solution to the ODE

input_signal_t = np.zeros(len(t))
for i in range(len(t)):
    input_signal_t[i] = input_signalODE(t[i])
    
plt.figure(5)
plt.plot(t,input_signal_t,label='input signal')
plt.plot(t,x,label='state')
plt.xlabel('time')
plt.legend()